In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [36]:
df = pd.read_csv(r'C:\Users\nirma\OneDrive\Desktop\fake-news\train.csv')

In [37]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [38]:
df.shape

(20800, 5)

In [39]:
df.drop_duplicates(inplace=True)
df.shape

(20800, 5)

In [40]:
df.dropna(axis=0,inplace=True)
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [41]:
df.shape

(18285, 5)

In [42]:
df['combines']=df['author']+' '+df['title']
df.head()

,id,title,author,text,label,combines
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nirma\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [43]:
def process_text(text):
    nopunc=[char for char in text if char not in string.punctuation]
    nopunc=''.join(nopunc)
    
    clear_word=[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clear_word

In [44]:
df['combines'].head().apply(process_text)

0    [Darrell, Lucus, House, Dem, Aide, Didn’t, Eve...
1    [Daniel, J, Flynn, FLYNN, Hillary, Clinton, Bi...
2        [Consortiumnewscom, Truth, Might, Get, Fired]
3    [Jessica, Purkiss, 15, Civilians, Killed, Sing...
4    [Howard, Portnoy, Iranian, woman, jailed, fict...
Name: combines, dtype: object

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
message_bow = CountVectorizer(analyzer=process_text).fit_transform(df['combines'])

In [46]:
message_bow.shape

(18285, 33610)

In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(message_bow,df['label'],test_size=0.2, random_state=0)

In [48]:
from sklearn.naive_bayes import MultinomialNB
classifier1=MultinomialNB()
classifier1.fit(X_train,y_train)

MultinomialNB()

In [49]:
print(classifier1.predict(X_train))
print(y_train.values)

[1 1 0 ... 1 0 0]
[1 1 1 ... 1 0 0]


In [50]:
from sklearn.metrics import classification_report
pred=classifier1.predict(X_test)
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      2040
           1       1.00      0.94      0.97      1617

    accuracy                           0.97      3657
   macro avg       0.98      0.97      0.97      3657
weighted avg       0.98      0.97      0.97      3657

